### Res Net

## demo

In [1]:
import torch 
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"


In [6]:
tensor = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]], dtype=torch.float64)
print(tensor.to(device))


tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]], device='cuda:0', dtype=torch.float64)


## actual

In [1]:
import torch 
import numpy as np
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"


ModuleNotFoundError: No module named 'sklearn'

In [8]:
print(device)


cuda
